## 修改caption_blur.json

In [ ]:
import json

# 读取原始JSON文件
with open('../artifacts/blur_dataset/captions_blur.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 修改每个样本
for sample in data:
    # sample["sharp_image"] = sample.pop("image_name")
    sample["motion_blur_image"] = sample["sharp_image"][:sample["sharp_image"].find("_S.")]+'_M.'+sample["sharp_image"][sample["sharp_image"].find("_S.")+len("_S."):]
    sample["defocused_blur_image"] = sample["sharp_image"][:sample["sharp_image"].find("_S.")]+'_F.'+sample["sharp_image"][sample["sharp_image"].find("_S.")+len("_S."):]

# 保存修改后的JSON文件
with open('../artifacts/blur_dataset/captions_blur.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
import json

# 读取原始JSON文件
with open('../artifacts/mpii/captions_mpii.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 修改每个样本
for sample in data:
    sample["label_0"] = 1

# 保存修改后的JSON文件
with open('../artifacts/mpii/captions_mpii.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

## 修改caption_human

In [ ]:
import json

# 读取原始JSON文件
with open('../artifacts/objects_indoor/captions_objects.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 修改每个样本
for sample in data:
    sample["image_0"]=sample.pop("image_name")

# 保存修改后的JSON文件
with open('../artifacts/objects_indoor/captions_objects.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
import os
import json

# 读取json文件
with open('../artifacts/mpii/captions_mpii.json', 'r') as f:
    data = json.load(f)

# 指定包含jpg文件的目录
jpg_directory = '../artifacts/mpii/examples_mpii'

# 遍历每条数据
for i, item in enumerate(data):
    # 获取image_name
    image_name = item['image_name']
    image_name = image_name.split('.')[0]
    
    # 在目录中查找含有image_name的文件名
    matched_files = [file for file in os.listdir(jpg_directory) if image_name in file]
    
    # 为每个匹配的文件名添加标签并保存到json中
    for j, matched_file in enumerate(matched_files):
        # 构建标签名称
        label_name = f'image_{j}'
        # 更新json数据
        data[i][label_name] = matched_file

# 将更新后的数据保存回json文件中
with open('../artifacts/mpii/your_updated_json_file.json', 'w') as f:
    json.dump(data, f, indent=4)



In [ ]:
import json

# Initialize an empty list to store extracted data
extracted_data = []

# Open the JSONL file for reading
with open('../safety/llava15_13b_inpainting_short_caption.jsonl', 'r') as file:
    for line in file:
        # Load each line as JSON
        data = json.loads(line)
        
        # Extract id and answer
        image_id = data['id']
        caption = data['answer']
        
        # Modify keys
        new_data = {'image_0': image_id[image_id.find('_')+len('_'):], 'image_1': image_id, 'caption': caption, 'label_0': 1}
        
        # Append the modified data to the list
        extracted_data.append(new_data)

# Save the extracted data to a JSON file
with open('../safety/extracted_data.json', 'w') as json_file:
    json.dump(extracted_data, json_file, indent=4)


### get pred

In [ ]:
import json
import os
def get_pred(prob_0, prob_1, threshold=0.0):
    if abs(prob_1 - prob_0) <= threshold:
        pred = "tie"
    elif prob_0 > prob_1:
        pred = "0"
    else:
        pred = "1"
    return pred


def get_label(example):
    if example["label_0"] == 0.5:
        label = "tie"
    elif example["label_0"] == 1:
        label = "0"
    else:
        label = "1"
    return label

base_dir = "./result/pickscore_v1"
file_names = ["mpii0.0.json", "nsfw0.0.json"]
for file_name in file_names:
    file_path = os.path.join(base_dir, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        
        
    for example in data:
        pred = get_pred(example["score_0"], example["score_1"])
        example["pred"] = pred
        
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, indent=4)

In [ ]:
import json
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from scipy.stats import pearsonr

pick_score_dir = "../result/acc/aesthetics/blur0.0.json"

with open(pick_score_dir, "r") as f:
    pick_score = json.load(f)

def get_pred(pred):
    if pred == "0":
        preference = 0
    elif pred == "1":
        preference = 1
    else:
        preference = 0.5
    return preference

human_preference = []
humanscore_preference = []
pickscore_preference = []

for pick_item in pick_score:
    pick_pred = pick_item["pred"]
    pick_preference = get_pred(pick_pred)
    pickscore_preference.append(pick_preference)
    human_pred = pick_item["label"]
    human_preference = get_pred(human_pred)
    humanscore_preference.append(human_preference)

print("humanscore_preference", len(humanscore_preference))
print("pickscore_preference", len(pickscore_preference))

print("Pick Preference Tie: ", pickscore_preference.count(0.5))
print("Human Preference Tie: ", humanscore_preference.count(0.5))

sum_all = 0
sum_valid = 0
for human, pick in zip(humanscore_preference, pickscore_preference):
    if human == 0.5 or pick == 0.5:
        continue
    elif human == pick:
        sum_all += 1
        sum_valid += 1
    else:
        sum_all += 1

print(f"filter out {len(humanscore_preference)-sum_all} samples")
print(f"Pick filter tie Accuracy: {sum_valid/sum_all}")

sum_all = 0
sum_valid = 0
sum_tie = 0
pick_filtered_score = []
for human, pick in zip(humanscore_preference, pickscore_preference):
    if human != 0.5:
        pick_filtered_score.append(pick)
    if human == 0.5:
        continue
    elif human == pick:
        sum_all += 1
        sum_valid += 1
    else:
        sum_all += 1
    
    if pick == 0.5:
        sum_tie += 1

print(f"re-introduce ties {sum_tie} samples")
print(f"re-introduce filter out {len(humanscore_preference)-sum_all} samples")
print(f"re-introduce Pick filter tie Accuracy: {sum_valid/sum_all}")

pearson_corr, _ = pearsonr(pick_filtered_score, humanscore_preference)
print(f"Pickscore Pearson Correlation Coefficient: {pearson_corr}")

kappa = cohen_kappa_score(pick_filtered_score, humanscore_preference)
print(f"Human-Pick Cohen's Kappa: {kappa}")

mse = mean_squared_error(pick_filtered_score, humanscore_preference)
print(f"Pick-Human MSE: {mse}")


### json2txt

In [5]:
import json

# 从JSON文件中读取数据
data_path = '../artifacts/objects_indoor/captions_objects.json'
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 创建一个txt文件来写入coco_caption
with open('artifacts_human_finetune_prompt.txt', 'a', encoding='utf-8') as f:
    for sample in data:
        caption = sample['caption']
        f.write(caption + '\n')
